# Supersingular Isogeny Diffie-Hellman

## Recherche d'un $p$ convenable

In [29]:
lA = 2
eA = 100

lB = 3
eB = 100

f = 1
p = lA ^ eA * lB ^ eB * f - 1
while f % lA == 0 or f % lB == 0 or p not in Primes():
    f += 1 
    p = lA ^ eA * lB ^ eB * f - 1

In [30]:
q = p^2
Fq = GF(q)

print("p : ", p)
print("Sécurité : ", numerical_approx(2*log(p)/log(2)))
print("Debug : f =", f)

p :  114330759112512408566920796752660118594000149332767102520037594114661999758540799
Sécurité :  531.894922367896
Debug : f = 175


## Recherche d'une courbe elliptique supersingulière de bonne cardinalité

In [31]:
E = EllipticCurve([Fq(0), Fq(1)])
print(E)
print("Supersingulière :", E.is_supersingular())
desiredCardinality = (lA ^ eA * lB ^ eB * f) ^ 2
print("Bonne cardinalité :", E.cardinality() == desiredCardinality)

Elliptic Curve defined by y^2 = x^3 + 1 over Finite Field in z2 of size 114330759112512408566920796752660118594000149332767102520037594114661999758540799^2
Supersingulière : True
Bonne cardinalité : True


## Recherche d'une base de la torsion

In [32]:
is_a_base = False
while not is_a_base:
    PA = 0
    while PA.order() != lA^eA: PA = (lB^eB * f) ^2 * E.random_point()
    QA = 0
    while QA.order() != lA^eA: QA = (lB^eB * f) ^2 * E.random_point()
    is_a_base = PA.weil_pairing(QA, lA^eA) ^ (lA^(eA-1)) != 1

is_a_base = False
while not is_a_base:
    PB = 0
    while PB.order() != lB^eB: PB = (lA^eA * f) ^2 * E.random_point()
    QB = 0
    while QB.order() != lB^eB: QB = (lA^eA * f) ^2 * E.random_point()
    is_a_base = PB.weil_pairing(QB, lB^eB) ^ (lB^(eB-1)) != 1

## Isogénies de degré friable

In [33]:
def chain_isogenies(E, R, l, e, points):
    phi = E.identity_morphism()
    Ei = E
    Ri = R
    for i in range(e):
        phi_i = Ei.isogeny(l^(e-i-1) * Ri)
        Ri = phi_i(Ri)
        Ei = phi_i.codomain()
        points = [phi_i(P) for P in points]
    return (Ei, points)

## Alice

In [34]:
def random_secret(l, e):
    badCouple = True
    while badCouple:
        m, n = randrange(l^e), randrange(l^e)
        badCouple = m % l == 0 and n % l == 0
    return (m, n)

In [35]:
mA, nA = random_secret(lA, eA)

(EA, [phiA_PB, phiA_QB]) = chain_isogenies(E, mA * PA + nA * QA, lA, eA, [PB, QB])

print("Secret :", mA, nA)
print("Publique :", EA)
print("Publique :", phiA_PB, phiA_QB)

Secret : 92713651467336170752671529405 930817743216827691882889413472
Publique : Elliptic Curve defined by y^2 = x^3 + (83551932065602948400424806201074340153098284438722542745689204430001980335364440*z2+64716305255802571726319403883796738277828114278442485300632929871102604084354977)*x + (16502019526401555534022593062060609280440493985721238259797290822882577527669549*z2+10901390390933334691237935986505881665071531054262283243654664061004044120939816) over Finite Field in z2 of size 114330759112512408566920796752660118594000149332767102520037594114661999758540799^2
Publique : (43968294715269403093398162348605149593287375965349801156546356709835097555945672*z2 + 79584676548827075208076365329934020550229863336937082068593281818067399494208606 : 83847247123963459242030428785469432703116044865694812518839180647983752542035377*z2 + 67685060528725147715138918047916109396440008615103693614224315101999219951007212 : 1) (8795771990981021439420634913605782958537579073425283597684302196282058927

## Bob

In [36]:
mB, nB = random_secret(lB, eB)

(EB, [phiB_PA, phiB_QA]) = chain_isogenies(E, mB * PB + nB * QB, lB, eB, [PA, QA])

print("Secret :", mB, nB)
print("Publique :", EB)
print("Publique :", phiB_PA, phiB_QA)

Secret : 358836851414785950867171116514324627376811280726 246746108481582123772028033833157818051923632492
Publique : Elliptic Curve defined by y^2 = x^3 + (26383646855277867872787058375031428649952267907927455766920913103918683677111748*z2+101257596811066808137927095041937034902522273615638951017755965372325674492513687)*x + (752495133336420119278872312176581004805886250808402047320869113326860019652421*z2+58586935735514478961616114730666706083701188271139895427461498006143729843891310) over Finite Field in z2 of size 114330759112512408566920796752660118594000149332767102520037594114661999758540799^2
Publique : (49007719929845248178076688781838088747485818224051842214402839484406188801897839*z2 + 8908576386373287101732190378606271610398255896930154400817826802996600365687910 : 20997843936006500924452434645771931516718926748297256998910222095504736387378527*z2 + 45951041588653654178874388665524493416971213761388880583481304496158895486156295 : 1) (10519027592964418527201908163054021500

## Alice

In [37]:
chain_isogenies(EB, mA * phiB_PA + nA * phiB_QA, lA, eA, [])[0].j_invariant()

102852686953988911347579561079604319569442450438651266947534680769953480140220081*z2 + 113220072002502050206209673736450004520070795545986011568658665430942398665549037

## Bob

In [38]:
chain_isogenies(EA, mB * phiA_PB + nB * phiA_QB, lB, eB, [])[0].j_invariant()

102852686953988911347579561079604319569442450438651266947534680769953480140220081*z2 + 113220072002502050206209673736450004520070795545986011568658665430942398665549037